In [2]:
import pandas as pd
df = pd.read_csv("../../combined_file.csv")
df.drop(['definition', 'column_name'], axis=1, inplace=True)
df.columns

C:\Users\Franc\AppData\Local\Temp\ipykernel_18960\2358656560.py:2: DtypeWarning: Columns (0,1,2,8,9,12,13,15,17,18,19,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../combined_file.csv")


Index(['most_recent_observation', 'most_recent_observation_type',
       'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
       'height_M', 'native', 'height_binned_M',
       'diameter_breast_height_binned_CM', 'greater_metro', 'city_ID',
       'tree_ID', 'planted_date', 'retired_date', 'location_type', 'zipcode',
       'neighborhood', 'location_name', 'ward', 'district', 'overhead_utility',
       'diameter_breast_height_CM', 'percent_population'],
      dtype='object')

In [10]:
df = df[["state", "city", "common_name"]]
df = df.dropna()
df = df[df["common_name"]!="ok to plant vacant"]
df

,state,city,common_name
0,New Mexico,Albuquerque,Locust
1,New Mexico,Albuquerque,Oak
2,New Mexico,Albuquerque,Ash
3,New Mexico,Albuquerque,Scotch pine
4,New Mexico,Albuquerque,Honeylocust
...,...,...,...
5642805,District of Columbia,Washington DC,Redmond american linden
5642806,District of Columbia,Washington DC,Dogwood
5642807,District of Columbia,Washington DC,Flowering dogwood
5642808,District of Columbia,Washington DC,(see notes)


In [11]:
# Count frecuency of every common_name in the dataset and select top 10
top_common_names = df['common_name'].value_counts().head(10).index

# Filter the dataframe
df1 = df[df['common_name'].isin(top_common_names)]
df1 = df1.groupby(['state', 'city', 'common_name']).size().reset_index(name='count')

df1.head(10)

,state,city,common_name,count
0,California,Anaheim,Crape myrtle,3320
1,California,Anaheim,Red maple,15
2,California,Fresno,Callery pear,55
3,California,Fresno,London planetree,294
4,California,Fresno,Pin oak,1
5,California,Fresno,Red maple,16
6,California,Garden Grove,Crape myrtle,2355
7,California,Garden Grove,Red maple,3
8,California,Huntington Beach,Crape myrtle,3813
9,California,Huntington Beach,Red maple,1


In [12]:
df1_sorted = df1.sort_values(by=['state', 'count'], ascending=[True, False])

# Preallocate of dataframe
df_result = pd.DataFrame()

# Iterate over each state and select top 3 cities with most sum
# Iterar sobre cada estado y seleccionar las 3 ciudades con la suma más alta
for state, group in df1_sorted.groupby('state'):
    top_cities = group.head(3)
    df_result = pd.concat([df_result, top_cities])

df_result.reset_index(drop=True, inplace=True)
df1 = df_result.copy()
df1

,state,city,common_name,count
0,California,Los Angeles,Crape myrtle,34729
1,California,Sacramento,London planetree,14111
2,California,Rancho Cucamonga,Crape myrtle,12877
3,Colorado,Denver,Green ash,25736
4,Colorado,Denver,Littleleaf linden,12961
...,...,...,...,...
65,Washington,Seattle,Norway maple,4649
66,Washington,Seattle,Callery pear,2540
67,Wisconsin,Madison,Norway maple,20918
68,Wisconsin,Madison,Green ash,17377


In [19]:
df1["state"].nunique()

24

In [13]:
df1["city"].nunique()
# To avoid circular link
df1["state"] = df1["state"].replace("New York", "New York State")

In [16]:
result = df1.copy()
result

,state,city,common_name,count
0,California,Los Angeles,Crape myrtle,34729
1,California,Sacramento,London planetree,14111
2,California,Rancho Cucamonga,Crape myrtle,12877
3,Colorado,Denver,Green ash,25736
4,Colorado,Denver,Littleleaf linden,12961
...,...,...,...,...
65,Washington,Seattle,Norway maple,4649
66,Washington,Seattle,Callery pear,2540
67,Wisconsin,Madison,Norway maple,20918
68,Wisconsin,Madison,Green ash,17377


In [18]:
result["state"].nunique()

24

In [17]:
# Convert to JSON and write it in a file
json_data = result.to_json(orient='records')
with open('../datasets/sankey.json', 'w') as file:
    file.write(json_data)

json_data

'[{"state":"California","city":"Los Angeles","common_name":"Crape myrtle","count":34729},{"state":"California","city":"Sacramento","common_name":"London planetree","count":14111},{"state":"California","city":"Rancho Cucamonga","common_name":"Crape myrtle","count":12877},{"state":"Colorado","city":"Denver","common_name":"Green ash","count":25736},{"state":"Colorado","city":"Denver","common_name":"Littleleaf linden","count":12961},{"state":"Colorado","city":"Aurora","common_name":"Honeylocust","count":8388},{"state":"District of Columbia","city":"Washington DC","common_name":"Red maple","count":12543},{"state":"District of Columbia","city":"Washington DC","common_name":"Pin oak","count":8815},{"state":"District of Columbia","city":"Washington DC","common_name":"Crape myrtle","count":4350},{"state":"Florida","city":"Orlando","common_name":"Live oak","count":17126},{"state":"Florida","city":"Cape Coral","common_name":"Live oak","count":5727},{"state":"Florida","city":"Tampa","common_name":